 <br><font color = darkblue size=6><strong>Black Litterman on ARK ETFs</strong></font> 

In [1]:
## Import libraries
# Import required libraries
import pandas as pd
import numpy as np
from numpy import *
from numpy.linalg import multi_dot

# Import yahoo finance libraries
import yfinance as yf

# Import plotly for visualization
import cufflinks as cf
import plotly.graph_objects as go


# Import optimization module from scipy
import scipy.optimize as sco


In [2]:
# Specify ARK ETFs
tickers = ['ARKK','ARKQ','ARKW','ARKG','ARKF','IZRL','PRNT','ACWI']

# Fetch the end of day price for entire history of data
ticker_prices = yf.download(tickers,progress=False)['Adj Close']

# we used 2019-12-31 as the starting point because that's the latest ETF (ARKQ) inception date
start_date='2019-12-31'
prices = ticker_prices[start_date:]

# normalized price series
# we used 2019-2-4 as the starting point because that's the latest ETF (ARKF) inception date
prices.normalize().iplot(title='ARK Invest ETFs Performance vs. MSCI ACWI')

In [3]:
# Resampling to derive weekly values from daily time series
prices_weekly = prices.resample('W').last()

# Use numpy log function to derive log normal returns
returns_weekly = np.log(prices_weekly).diff().fillna(0)
returns_annualized = returns_weekly.mean()*52

# calculate tracking error
excess = returns_weekly[['ARKK','ARKQ','ARKW','ARKG','ARKF','IZRL','PRNT']].subtract(returns_weekly['ACWI'],axis=0)
tr = (excess.std())*np.sqrt(52)
tr.iplot(kind='bar',title='Annualized Tracking Error vs MSCI ACWI')

In [4]:
# calcuate vol
vols_weekly = returns_weekly.std()
vols_annualized = vols_weekly*np.sqrt(52)

## Draw efficient frontier
# specify inputs
numofasset = len(tickers)


def portfolio_stats(weights):
    
    weights = array(weights)[:,newaxis]
    port_rets = weights.T @ array(returns_weekly.mean() * 52)[:,newaxis]    
    port_vols = sqrt(multi_dot([weights.T, returns_weekly.cov() * 52, weights])) 
    
    return np.array([port_rets, port_vols, port_rets/port_vols]).flatten()

bnds = tuple((0, 1) for x in range(numofasset))
initial_wts = numofasset*[1./numofasset]

# Minimize the volatility
def min_volatility(weights):
    return portfolio_stats(weights)[1]

targetrets = linspace(returns_annualized.min(),returns_annualized.max(),100)
tvols = []

for tr in targetrets:
    
    ef_cons = ({'type': 'eq', 'fun': lambda x: portfolio_stats(x)[0] - tr},
               {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    
    opt_ef = sco.minimize(min_volatility, initial_wts, method='SLSQP', bounds=bnds, constraints=ef_cons)
    
    tvols.append(opt_ef['fun'])

targetvols = array(tvols)

# Create a dataframe for plotting
efrontier = pd.DataFrame({'returns': targetrets,
                      'volatility': targetvols})


In [5]:
#plot efficient frontier
fig=go.Figure(data=[go.Scatter(name='Efficient Frontier',x=targetvols,y=targetrets),
                   go.Scatter(name ='Tickers',x=vols_annualized,y=returns_annualized, mode='markers', marker_symbol='star',
                             marker_size=20, hovertemplate= vols_annualized.index)])
fig.update_layout(title='Opportunity Set', xaxis_title='Volatility',yaxis_title='Return')
fig.show()

# Building out the Portfolio

For the portfolio, we start by assuming all is in ACWI.

We then express our views in terms of excess alpha we can get from thematic investing vs. ACWI and see how the allocation would change.

In [6]:
import numpy as np
import scipy as sp
import scipy.stats as stat
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interact, FloatSlider

In [7]:
# Sample size
T = 10000
N = numofasset

# random market portfolio (sum is normalized to 1)
#w_m = np.random.rand(N)
w_m = np.array([1,0,0,0,0,0,0,0]).reshape(-1,1)
#w_m = np.array([0.90,0.1/7,0.1/7,0.1/7,0.1/7,0.1/7,0.1/7,0.1/7])
w_m = w_m / (w_m.sum())

# Generate a sample of returns
returns = stat.multivariate_normal(returns_weekly.mean(), returns_weekly.cov())
np.random.seed(seed=1)
sample = returns.rvs(T)

# Estimate μ and Σ
mu_est = (sample*52).mean(0).reshape(N, 1)
cov_est = np.cov(sample.T)*52


# Observed mean market return
r_m = w_m.T @ mu_est

# Estimated variance of the market portfolio
var_m = w_m.T @ cov_est @ w_m

# Sharpe-ratio
sr_m = r_m / np.sqrt(var_m)

# Risk aversion of market portfolio holder, lambda
d_m = sr_m / np.sqrt(var_m)
#d_m=0.1
# Derive "view" which would induce the market portfolio
mu_m = (d_m * cov_est @ w_m).reshape(N, 1)

# set up position matrix P
P = np.array([[-1,1,0,0,0,0,0,0],
            [-1,0,1,0,0,0,0,0],
            [-1,0,0,1,0,0,0,0],
            [-1,0,0,0,1,0,0,0],
            [-1,0,0,0,0,1,0,0],
            [-1,0,0,0,0,0,1,0],
            [-1,0,0,0,0,0,0,1]])

# calcualte tau
tau = 1/T

# set up view matrix Q
Q3 = np.full((len(tickers)-1, 1), 0.03)
Q10 = np.full((len(tickers)-1, 1), 0.10)

# set up confidence matrix Omega
Omega = np.diag(np.diag(np.linalg.multi_dot([P,tau*cov_est,P.T])))
#Omega = np.array([0.95,0.95,0.95,0.95,0.95,0.95,0.95])
#Omega = np.diag(Omega)

In [8]:
def black_litterman(λ, μ1, μ2, Σ1, Σ2):
    """
    This function calculates the Black-Litterman mixture
    mean excess return and covariance matrix
    """
    Σ1_inv = np.linalg.inv(Σ1)
    Σ2_inv = np.linalg.inv(Σ2)

    μ_tilde = np.linalg.solve(Σ1_inv + λ * Σ2_inv,
                              Σ1_inv @ μ1 + λ * Σ2_inv @ μ2)
    return μ_tilde

def black_litterman_meucci(tau,pi, cov, p, omega, q):
    """
    This function calculates the Black-Litterman mixture
    mean excess return and covariance matrix
    """
    matrix1=np.linalg.inv(tau*cov) + np.linalg.multi_dot([p.T,np.linalg.inv(omega),p])
    matrix2=np.dot(np.linalg.inv(tau*cov),pi) + np.linalg.multi_dot([p.T,np.linalg.inv(omega),q])
    
    μ_tilde = np.dot(np.linalg.inv(matrix1),matrix2)
    return μ_tilde



#mu_tilde = black_litterman(1,mu_m, mu_est,cov_est, tau*cov_est)
mu_tilde3 = black_litterman_meucci(tau, mu_m, cov_est, P, Omega, Q3)
mu_tilde10 = black_litterman_meucci(tau, mu_m, cov_est, P, Omega, Q10)


# The Black-Litterman recommendation for the portfolio weights
w_tilde3 = (1/d_m)*np.dot(np.linalg.inv(cov_est),mu_tilde3)
w_tilde10 = (1/d_m)*np.dot(np.linalg.inv(cov_est),mu_tilde10)

In [9]:
## plot allocation based on 3% vs 10% outperformance
# create a dataframe for plotting
allocation = pd.DataFrame({'Original':w_m.flatten(),
                        '3% View': w_tilde3.flatten(),
                        '10% View':w_tilde10.flatten()},index=returns_annualized.index)

allocation.iplot(kind='bar', title='Allocation to ACWI and ARK ETFs')

In [10]:
w_sate = np.array([w_m.flatten(),w_tilde3.flatten(),w_tilde10.flatten()])
mu_sate = np.array([mu_m.flatten(),mu_tilde3.flatten(),mu_tilde10.flatten()])

returns_sate = np.diag(np.dot(w_sate,mu_sate.T))
vols_sate = np.diag(sqrt(np.linalg.multi_dot([w_sate,cov_est,w_sate.T])))

returnrisk_sate = pd.DataFrame({'Portfolio':['Original','3% View','10% View'],
                                'Return': returns_sate,
                               'Volatility': vols_sate})

returnrisk_sate.iplot(kind='scatter',categories = 'Portfolio',
                      y='Return',x='Volatility', mode='markers',
                     title = 'Return Risk Profile',xTitle ='Risk', yTitle='Return')

C:\Users\Grace\anaconda3\lib\site-packages\cufflinks\plotlytools.py:807: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead

C:\Users\Grace\anaconda3\lib\site-packages\cufflinks\plotlytools.py:810: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead

